In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Abrir los archivos .csv como un DataFrame
# Importar pandas y numpy para usarlos como pd y np respectivamente
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import pyplot
import seaborn as sns
from pandas.plotting import scatter_matrix
import scikitplot as skplt

In [ ]:
#importing the data
data = pd.read_csv("../input/spotify-dataset-19212020-160k-tracks/tracks.csv")
data

In [ ]:
data.tail()

In [ ]:
### 1) Pre-Procesado

In [ ]:
# muestra la informacion que contiene nuestro dataset
print(data.info())

In [ ]:
#Limpìeza : Busqueda de valores vacios
data.isnull().any()

In [ ]:
#limpieza : Suma de valores vacios
data.isnull().sum()

In [ ]:
data.duplicated().sum()


In [ ]:
nan_rows = data[data.isnull().any(1)]
print(nan_rows)

In [ ]:
data.drop(data[data.name.isnull()].index, inplace = True)

In [ ]:
data.isnull().sum()

In [ ]:
#Hay ciertas columnas que no necesitamops ya que no aportan información relevante para este proyecto
data.drop(columns=['id_artists','id','time_signature'], inplace=True)

In [ ]:
data.shape

In [ ]:
data['release_date'] = pd.to_datetime(data['release_date'])

In [ ]:
data.describe()

# 2) EDA

# Popularity

Popularity:
La popularidad de las canciones. El valor estará entre 0 y 100, siendo 100 el más popular.

La popularidad se calcula a partir de la popularidad de las pistas individuales del álbum.

El desarrollo de este proyecto se va a enfocar en la popularidad de las canciones a partir de las caracteristicas de estas.

La popularidad se calcula mediante un algoritmo y se basa, en su mayor parte, en el número total de reproducciones que ha tenido la pista y lo recientes que son esas reproducciones.

In [ ]:
#Vamos a crear una nueva columna de año para facilitar las visualizaciones
data['year'] = data.apply(lambda row: row.release_date.year, axis = 1)

In [ ]:
años = data[["acousticness","danceability","energy", "instrumentalness", 
               "liveness","tempo", "valence", "loudness", "speechiness", "year","popularity"]].\
groupby("year").mean().sort_values(by="year").reset_index()

# year_avg.head()
plt.figure(figsize=(14,8))
plt.title("Tendencia de popularidad de canciones a lo largo del tiempo", fontdict={"fontsize": 15})

lines = ["popularity"]

for line in lines:
    ax = sns.lineplot(x='year', y=line, data=años)
    
    
plt.ylabel("Popularity")
plt.legend(lines)

# Explicit

Explicit :
Los contenidos explícitos suelen llevar la etiqueta Parental Advisory (abreviada PAL), que es una etiqueta de advertencia.

Se coloca en las grabaciones de audio en reconocimiento de blasfemias o referencias inapropiadas, con la intención de alertar a los padres sobre el material potencialmente inadecuado para los niños.

La etiqueta se ha incluido en las listas digitales que ofrecen las tiendas de música en línea (como Spotify).

Los valores de la caracteristica 'Explicit' van entre 0 si no es explicito y 1 en el caso de que lo sea

La correlación lineal con la popularidad es de 0.2117

In [ ]:
data['explicit'].corr(data['popularity'])

In [ ]:
exp_data = sns.catplot(data=data, kind="bar",
                        x="explicit", y="popularity",
                        ci="sd", palette='Accent',
                        height=6)
exp_data.fig.suptitle('Popularity vs. Explicit', y=1.05, c='r')
exp_data.set_axis_labels("", "Popularity")
exp_data.set_xticklabels(['No Explicit\nContent', 'Explicit\nContent'])
plt.show()

# Key

In [ ]:
data['key'].corr(data['popularity'])

In [ ]:
exp_data = sns.catplot(data=data, kind="bar",
                        x="key", y="popularity",
                        palette='pastel',
                        ci = 'sd',
                        height=5, aspect = 3)
exp_data.fig.suptitle('Popularity by Key Category', y=1.05, c='r', weight='bold')
exp_data.set_axis_labels("key", "Popularity")
plt.show()

# Mode

El modo indica la modalidad (mayor o menor) de una pista, el tipo de escala del que se deriva su contenido melódico.

El mayor se representa con 1 y el menor con 0.

In [ ]:
data['mode'].corr(data['popularity'])

In [ ]:
exp_data = sns.catplot(data=data, kind="bar",
                        x="mode", y="popularity",
                        ci="sd", palette='Accent',
                        height=4)
exp_data.fig.suptitle('Popularity vs. Mode', y=1.05, c='r')
exp_data.set_axis_labels("", "Popularity")
exp_data.set_xticklabels(['Minor (0)', 'Major(1)'])
plt.show()

# Acousticness

Una medida de confianza de 0,0 a 1,0 sobre si la pista es acústica. 1,0 representa una alta confianza en que la pista es acústica.

La acústica de la mayoría de las pistas es cercana a 0 o 1

Cuanto más se confíe en que un tema es acústico, menor será su popularidad media (se aplica para acousitcness > 0,1)

In [ ]:
data['acousticness'].corr(data['popularity'])

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))
ax1_data =  data.groupby('acousticness')['popularity'].mean().to_frame().reset_index()
ax = sns.scatterplot(x = ax1_data['acousticness'], y = ax1_data['popularity'], color='blue', ax=ax)
ax.set_title('Acousticness vs. Mean Popularity')
ax.set_ylabel('Mean Popularity', fontsize=12)
plt.tight_layout()
plt.show()

# Danceabilitiy

La bailabilidad describe la idoneidad de una pista para el baile basándose en una combinación de elementos musicales como el tempo, la estabilidad del ritmo, la fuerza del compás y la regularidad general. Un valor de 0,0 es el menos bailable y 1,0 el más bailable.

In [ ]:
data['danceability'].corr(data['popularity'])

In [ ]:
fig, ax = plt.subplots(1, figsize=(15, 6), sharey=True, sharex = True)
ax_data =  data.groupby('danceability')['popularity'].mean().to_frame().reset_index()
ax = sns.scatterplot(x='danceability', y='popularity', data=ax_data, color='blue', ax=ax)
ax.set_title('Danceability vs. Mean Popularity')
ax.set_ylabel('Mean Popularity', fontsize=12)
plt.tight_layout()
plt.show()

# Energy

La energía es una medida de 0,0 a 1,0 y representa una medida perceptiva de intensidad y actividad. Por lo general, las pistas energéticas se sienten rápidas, ruidosas y con mucho ruido. Por ejemplo, el death metal tiene mucha energía, mientras que un preludio de Bach tiene una puntuación baja en la escala.

Entre las características perceptivas que contribuyen a este atributo se encuentran el rango dinámico, el volumen percibido, el timbre, la velocidad de aparición y la entropía general

In [ ]:
data['energy'].corr(data['popularity'])

In [ ]:
fig, ax = plt.subplots(1, figsize=(15, 6), sharey=True, sharex = True)
ax_data =  data.groupby('energy')['popularity'].mean().to_frame().reset_index()
ax = sns.scatterplot(x='energy', y='popularity', data=ax_data, color='blue', ax=ax)
ax.set_title('Energy vs. Mean Popularity')
ax.set_ylabel('Mean Popularity', fontsize=12)
plt.tight_layout()
plt.show()

# Instrumentalness

Predice si una pista no contiene voces. Los sonidos "Ooh" y "aah" se consideran instrumentales en este contexto. Las pistas de rap o de palabras habladas son claramente "vocales".

Cuanto más se acerque el valor de instrumentalización a 1,0, mayor será la probabilidad de que la pista no tenga contenido vocal. Los valores superiores a 0,5 representan pistas instrumentales, pero la confianza es mayor a medida que el valor se acerca a 1,0

In [ ]:
data['instrumentalness'].corr(data['popularity'])

In [ ]:
fig, ax = plt.subplots(1, figsize=(15, 6), sharey=True, sharex = True)
ax_data =  data.groupby('instrumentalness')['popularity'].mean().to_frame().reset_index()
ax = sns.scatterplot(x='instrumentalness', y='popularity', data=ax_data, color='blue', ax=ax)
ax.set_title('Instrumentalness vs. Mean Popularity')
ax.set_ylabel('Mean Popularity', fontsize=12)
plt.tight_layout()
plt.show()

# Liveness

Detecta la presencia de público en la grabación. Los valores más altos de liveness representan una mayor probabilidad de que la pista haya sido interpretada en directo.

Un valor superior a 0,8 proporciona una fuerte probabilidad de que la pista sea en directo

In [ ]:
data['liveness'].corr(data['popularity'])

In [ ]:
fig, ax = plt.subplots(1, figsize=(15, 6), sharey=True, sharex = True)
ax_data =  data.groupby('liveness')['popularity'].mean().to_frame().reset_index()
ax = sns.scatterplot(x='liveness', y='popularity', data=ax_data, color='blue', ax=ax)
ax.set_title('Liveness vs. Mean Popularity')
ax.set_ylabel('Mean Popularity', fontsize=12)
plt.tight_layout()
plt.show()

# Loudness

La sonoridad general de una pista en decibelios (dB). Los valores de sonoridad se promedian en toda la pista y son útiles para comparar la sonoridad relativa de las pistas.

La sonoridad es la cualidad de un sonido que es el principal correlato psicológico de la fuerza física (amplitud). Los valores suelen oscilar entre -60 y 0 db.

In [ ]:
data['loudness'].corr(data['popularity'])

In [ ]:
fig, ax = plt.subplots(figsize = (15, 6))
sns.scatterplot(x='loudness', y='popularity', data=data, color='blue', alpha=0.3)
plt.show()

# Speechiness

La expresividad detecta la presencia de palabras habladas en una pista. Cuanto más exclusivamente hablada sea la grabación (por ejemplo, un programa de entrevistas, un audiolibro o una poesía), más se acercará a 1,0 el valor del atributo.

Los valores superiores a 0,66 describen pistas que probablemente estén compuestas exclusivamente por palabras habladas.

Los valores entre 0,33 y 0,66 describen pistas que pueden contener tanto música como voz, ya sea en secciones o en capas, incluyendo casos como la música rap. Los valores inferiores a 0,33 representan probablemente música y otras pistas no habladas.



In [ ]:
data['speechiness'].corr(data['popularity'])

In [ ]:
fig, ax = plt.subplots(1, figsize=(15, 6), sharey=True, sharex = True)
ax_data =  data.groupby('speechiness')['popularity'].mean().to_frame().reset_index()
ax = sns.scatterplot(x='speechiness', y='popularity', data=ax_data, color='blue', ax=ax)
ax.set_title('speechiness vs. Mean Popularity')
ax.set_ylabel('Mean Popularity', fontsize=12)
plt.tight_layout()
plt.show()

# Tempo

El tempo global estimado de una pista en pulsaciones por minuto (BPM).

En la terminología musical, el tempo es la velocidad o el ritmo de una pieza determinada y se deriva directamente de la duración media de los tiempos.

In [ ]:
data['tempo'].corr(data['popularity'])

In [ ]:
fig, ax = plt.subplots(1, figsize=(15, 6), sharey=True, sharex = True)
ax_data =  data.groupby('tempo')['popularity'].mean().to_frame().reset_index()
ax = sns.scatterplot(x='tempo', y='popularity', data=ax_data, color='blue', ax=ax)
ax.set_title('tempo vs. Mean Popularity')
ax.set_ylabel('Mean Popularity', fontsize=12)
plt.tight_layout()
plt.show()

# Valence

Una medida de 0,0 a 1,0 que describe la positividad musical que transmite una pista.

Las pistas con alta valencia suenan más positivas (por ejemplo, felices, alegres, eufóricas), mientras que las pistas con baja valencia suenan más negativas (por ejemplo, tristes, deprimidas, enfadadas).

In [ ]:
data['valence'].corr(data['popularity'])

In [ ]:
fig, ax = plt.subplots(1, figsize=(15, 6), sharey=True, sharex = True)
ax_data =  data.groupby('valence')['popularity'].mean().to_frame().reset_index()
ax = sns.scatterplot(x='valence', y='popularity', data=ax_data, color='blue', ax=ax)
ax.set_title('valence vs. Mean Popularity')
ax.set_ylabel('Mean Popularity', fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize = (15, 6))
sns.scatterplot(x='valence', y='popularity', data=data, color='blue', alpha=0.3)
plt.show()

## 3) Visualización 

In [ ]:
#Histogramas
data.hist(figsize=(20, 20))
plt.show()

In [ ]:
plt.figure(figsize = (40, 20))
plt.subplot(331)
sns.boxplot(data['danceability'])
plt.subplot(332)
sns.boxplot(data['duration_ms'])
plt.subplot(333)
sns.boxplot(data['energy'])
plt.subplot(334)
sns.boxplot(data['liveness'])
plt.subplot(335)
sns.boxplot(data['loudness'])
plt.subplot(336)
sns.boxplot(data['instrumentalness'])
plt.show()

In [ ]:
plt.figure(figsize=(16, 8))
sns.set(style="whitegrid")
corr = data.corr()
sns.heatmap(corr,annot=True)

In [ ]:
# # Plotting
fig, ax = plt.subplots(figsize = (12, 10))
lead_artists = data.groupby('artists')['popularity'].sum().sort_values(ascending=False).head(20)
ax = sns.barplot(x=lead_artists.values, y=lead_artists.index, palette="Blues", orient="h", edgecolor='black', ax=ax)
ax.set_xlabel('Sum of Popularity', c='r', fontsize=12)
ax.set_ylabel('Artist', c='r', fontsize=12)
ax.set_title('20 Most Popular Artists in Dataset', c='r', fontsize=14, weight = 'bold')
plt.show()

In [ ]:
# find out artists with more than 100 tracks
data['popular_artist'] = data['artists'].map(data['artists'].value_counts()>100)
pop_arts  = data.groupby(['artists', 'popular_artist'])['popularity'].mean().sort_values(ascending=False).reset_index(1)
df_pop_arts = pop_arts.loc[pop_arts['popular_artist'] == True,['popularity']]

# # Plotting
fig, ax = plt.subplots(figsize = (12, 10))
lead_artists = df_pop_arts.groupby('artists')['popularity'].mean().sort_values(ascending=False).head(20)
ax = sns.barplot(x=lead_artists.values, y=lead_artists.index, palette="Blues", orient="h", edgecolor='black', ax=ax)
ax.set_xlabel('Mean of Popularity', c='r', fontsize=12)
ax.set_ylabel('Artist', c='r', fontsize=12)
ax.set_title('20 Most Popular Artists in Dataset with > 100 tracks', c='r', fontsize=14, weight = 'bold')
plt.show()


In [ ]:
plt.figure(figsize=(16, 4))
sns.set(style="whitegrid")
x = data.groupby("name")["popularity"].mean().sort_values(ascending=False).head(20)
axis = sns.barplot(x.index, x)
axis.set_title('Top Tracks with Popularity')
axis.set_ylabel('Popularity')
axis.set_xlabel('Tracks')
plt.xticks(rotation = 90)


In [ ]:
plt.figure(figsize=(16, 4))
sns.set(style="whitegrid")
x = data.groupby("artists")["popularity"].sum().sort_values(ascending=False).head(20)
ax = sns.barplot(x.index, x)
ax.set_title('Top Artists with Popularity')
ax.set_ylabel('Popularity')
ax.set_xlabel('Artists')
plt.xticks(rotation = 90)

# 3) Modelado

In [ ]:
data.value_counts('popularity')

#mean_df = data['popularity'].mean()->27.5732124561666967

In [ ]:
data.loc[data['popularity'] < 30, 'popularity'] = 0 
data.loc[data['popularity'] >= 30, 'popularity'] = 1
data.loc[data['popularity'] == 1]

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC, LinearSVC


from sklearn.metrics import make_scorer, accuracy_score, roc_auc_score 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

features = ["acousticness","danceability","duration_ms","energy","explicit","instrumentalness"
            ,'key','liveness','loudness','mode','speechiness','tempo','valence','year']
training = data.sample(frac = 0.8,random_state = 420)
X_train = training[features]
y_train = training['popularity']
X_test = data.drop(training.index)[features]

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.2, random_state = 420)

In [ ]:
RFC_Model = RandomForestClassifier()
RFC_Model.fit(X_train, y_train)
RFC_Predict = RFC_Model.predict(X_valid)
RFC_Accuracy = accuracy_score(y_valid, RFC_Predict)
print("Accuracy: " + str(RFC_Accuracy))

RFC_AUC = roc_auc_score(y_valid, RFC_Predict) 
print("AUC: " + str(RFC_AUC))

In [ ]:
LR_Model = LogisticRegression()
LR_Model.fit(X_train, y_train)
LR_Predict = LR_Model.predict(X_valid)
LR_Accuracy = accuracy_score(y_valid, LR_Predict)
print("Accuracy: " + str(LR_Accuracy))

LR_AUC = roc_auc_score(y_valid, LR_Predict) 
print("AUC: " + str(LR_AUC))

In [ ]:
# AdaBoost Algorithm
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
model = AdaBoostClassifier()
# n_estimators = 50 (default value) 
# base_estimator = DecisionTreeClassifier (default value)
model.fit(X_train,y_train)
preds = model.predict(X_valid)
prediccion=accuracy_score(y_valid, preds)

print("Accuracy: " + str(prediccion))
ADA_AUC = roc_auc_score(y_valid, preds) 
print("AUC: " + str(ADA_AUC))from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
model = XGBClassifier()
model.fit(X_train, y_train)
preds = model.predict(X_valid)
prediccion=accuracy_score(y_valid, preds)

print("Accuracy: " + str(prediccion))
XG_AUC = roc_auc_score(y_valid, preds) 
print("AUC: " + str(XG_AUC))

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
model = XGBClassifier()
model.fit(X_train, y_train)
preds = model.predict(X_valid)
prediccion=accuracy_score(y_valid, preds)

print("Accuracy: " + str(prediccion))
XG_AUC = roc_auc_score(y_valid, preds) 
print("AUC: " + str(XG_AUC))

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_valid,preds))

In [ ]:
matrix=confusion_matrix(y_valid,preds)
print(matrix)